In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from collections import Counter

from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 90)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42 

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Подгрузим необходимые для работы файлы

In [ ]:
DATA_DIR = '/kaggle/input/restaurant-rating-prediction/'
df_train = pd.read_csv(DATA_DIR+'main_task.csv')
#df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
df_cit = pd.read_csv('/kaggle/input/world-cities/worldcities.csv')
df_cost=pd.read_csv('/kaggle/input/2020-cost-of-living/cost of living 2020.csv')

### Посмотрим на все файлы, чтобы представлять, с чем мы будем оперировать

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.columns

In [ ]:
#Приведем к более привычному для меня виду
df_train.rename(columns={'Price Range' : 'Price_Range',
                     'Cuisine Style':'Cuisine_Style',
                    'Number of Reviews':'Num_of_Revs'}, inplace=True)

In [ ]:
#df_test.head()

In [ ]:
#df_test.info()

In [ ]:
#df_test.isna().sum()

In [ ]:
df_cit.head()

In [ ]:
df_cit.info()

In [ ]:
df_cit.isna().sum()

In [ ]:
df_cost.head()

In [ ]:
df_cost.isna().sum()

In [ ]:
#df_train['sample'] = 1 # помечаем где у нас трейн
#df_test['sample'] = 0 # помечаем где у нас тест
#df_test['Rating'] = 0
#df_train = df_test.append(df_train, sort=False).reset_index(drop=True)

In [ ]:
df_train.info()

##  Restaurant_id

In [ ]:
df_train.Restaurant_id.value_counts()

## City

In [ ]:
df_train.City.unique()

In [ ]:
df_train.City.nunique()

In [ ]:
cityes=set()
df_train['City'].apply(lambda x: cityes.add(x))
cityes
len(cityes)

In [ ]:
df_cit.columns

In [ ]:
# Уберем лишние колонки, а так же заметим, что европейские города как-то попали в столбец country в United States, 
#это надо будет исправить
df_cit = df_cit.drop(['city_ascii', 'lat', 'lng', 'iso2',
                      'iso3', 'admin_name', 'capital', 'id'], axis='columns')
df_cit = df_cit.loc[df_cit.city.isin(cityes)]
df_cit

In [ ]:
df_cit=df_cit.loc[df_cit['country']!='United States']
new_city=set()
df_cit.city.apply(lambda x: new_city.add(x))

#Все ли города совпадают? Проверим
cityes-new_city 

In [ ]:
#Указанных выше городов нет, добавим их вручную
df_cit.loc[2586]=['Krakow', 'Poland',779115]
df_cit.loc[2587]=['Oporto', 'Portugal',240000]
df_cit.loc[2588]=['Zurich', 'Germany',1300000]
df_cit.tail(5)

In [ ]:
df_cost.head()

In [ ]:
#Уберем часть лишних колонок, оставим индекс цен в ресторанах, индекс бигмака, Индекс местной покупательной способности
df_cost = df_cost.drop(['Rank 2020', 'Cost of Living Index', 'Cost of Living Plus Rent Index', 'Groceries Index',
                        'Unnamed: 9', 'Rent Index'], axis='columns')
df_cost = df_cost.loc[df_cost.Country.isin(df_cit['country'])]
df_cost.sample(5)

In [ ]:
df_cost.rename(columns={'Restaurant Price Index':'Rest_pr_index', 'Local Purchasing Power Index':'Loc_Purch_Pow'}, inplace=True)

In [ ]:
df_cost.head()

In [ ]:
#Объединим два небольших датафрейма между собой
df_cit=df_cit.merge(df_cost, left_on='country',right_on= 'Country', how='inner')
df_cit=df_cit.drop(['country','Country'], axis='columns')
df_cit.head(5)

In [ ]:
# Объединим теперь с большим фреймом
df_train=df_train.merge(df_cit, left_on='City',right_on= 'city', how='inner')
df_train.head(5)

## Cuisine Style

In [ ]:
df_train['Cuisine_Style']=df_train['Cuisine_Style'].fillna("['no_info']") #заменяем пропуск 
def make_a_list (x):
    x=x[1:-1]
    x=x.replace("'", "")
    x=x.split(', ')
    return (x)
cousine_list=[]
df_train['Cuisine_Style']=df_train['Cuisine_Style'].apply(make_a_list)
df_train['Cuisine_Style'].apply(lambda x: cousine_list.extend(x))
Counter(cousine_list).most_common(3)

In [ ]:
def change_cuisine(x):
    if x == ['no_info']:
        return ['European', 'Vegetarian Friendly']
    else:
        return x
df_train['Cuisine_Style']=df_train['Cuisine_Style'].apply(change_cuisine)
cousine_list=[]
df_train['Cuisine_Style'].apply(lambda x: cousine_list.extend(x))
Counter(cousine_list).most_common(5)


In [ ]:
df_train['Cuisine_Style']=df_train['Cuisine_Style'].apply(lambda x: len(x))
df_train.head()

## Price Range

In [ ]:
df_train.Price_Range.value_counts()

In [ ]:
df_train['Price_Range']=df_train['Price_Range'].fillna('$$ - $$$')

In [ ]:
def replace_price(x):
    if x == "$$ - $$$":
        return 'medium price'
    elif x == "$$$$":
        return 'high price'
    elif x == "$":
        return 'low price'

In [ ]:
df_train['Price_Range'] = df_train['Price_Range'].apply(replace_price)

In [ ]:
data_price = pd.get_dummies(df_train['Price_Range'])
df_train['high price'] = data_price['high price']
df_train['low price'] = data_price['low price']
df_train['medium price'] = data_price['medium price']

In [ ]:
def replace_price_to_value(x):
    if x == "medium price":
        return '2'
    elif x == "high price":
        return '3'
    elif x == "low price":
        return '1'

In [ ]:
df_train['Price_Range'] = df_train['Price_Range'].apply(replace_price_to_value)

In [ ]:
df_train['Price_Range'].dropna(inplace=True)

Как-то очень хитро заверуто получилось, но по ходу анализа приходили разные мысли, поэтому пришлось ставить "костыль"

In [ ]:
df_train.head()

## Number of Reviews

In [ ]:
df_train['Num_of_Revs'].value_counts()

In [ ]:
df_train['Num_of_Revs'].hist(bins=100)
df_train['Num_of_Revs'].describe()

In [ ]:
mode=df_train['Num_of_Revs'].mode()
mean=df_train['Num_of_Revs'].mean()
median=df_train['Num_of_Revs'].median()

In [ ]:
print("Mode: {}, \n\nMean: {},\nMedian: {}".format(mode,mean,median))

In [ ]:
median = df_train['Num_of_Revs'].median()
IQR = df_train['Num_of_Revs'].quantile(0.75) - df_train['Num_of_Revs'].quantile(0.25)
perc25 = df_train['Num_of_Revs'].quantile(0.25)
perc75 = df_train['Num_of_Revs'].quantile(0.75)
print('25-й перцентиль: {},'.format(perc25), '75-й перцентиль: {},'.format(perc75)
      , "IQR: {}, ".format(IQR),"Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))
df_train['Num_of_Revs'].loc[df_train['Num_of_Revs'].between(perc25 - 1.5*IQR, perc75 + 1.5*IQR)].hist(bins = 50, range = (0, 40), 
                                                                                             label = 'IQR')
plt.legend();

In [ ]:
df_train['Num_of_Revs'] = df_train['Num_of_Revs'][df_train['Num_of_Revs'].between(perc25 - 1.5*IQR, perc75 + 1.5*IQR)]

In [ ]:
mean_val = round(df_train['Num_of_Revs'].mean(), 0)
df_train['Num_of_Revs'] = df_train['Num_of_Revs'].fillna(mean_val)

In [ ]:
df_train['Num_of_Revs'].hist()
df_train['Num_of_Revs'].describe()

## Reviews

In [ ]:
for i, cell in enumerate(df_train['Reviews']):
    print(cell)
    if i == 20:
        break

In [ ]:
df_train['Reviews'].fillna('', inplace=True)

In [ ]:
count = 0
for i in df_train['Reviews']:
    if i == '[[], []]':
        count +=1
print(count)

In [ ]:
count = 0
for i in df_train['Reviews']:
    if i == '[[], []]':
        df_train.drop(index = count, inplace = True, axis = 0)
    count +=1

In [ ]:
df_train['Reviews'].isna().sum()

In [ ]:
df_train['Reviews_date_1'] = df_train['Reviews'].apply(lambda x: re.findall(r'\d\d\/\d\d\/\d{4}', x))

In [ ]:
df_train['Reviews_date_2'] = df_train['Reviews_date_1']

In [ ]:
df_train['Reviews_date_1'] = df_train['Reviews_date_1'].apply(lambda x: x[0])

In [ ]:
df_train['Reviews_date_2'] = df_train['Reviews_date_2'].apply(
    lambda x: x[1] if len(x) > 1 else x[0])

In [ ]:
df_train.head()

In [ ]:
df_train['Reviews_date_1'] = df_train['Reviews_date_1'].apply(
    lambda x: datetime.strptime(x, '%m/%d/%Y'))
df_train['Reviews_date_2'] = df_train['Reviews_date_2'].apply(
    lambda x: datetime.strptime(x, '%m/%d/%Y'))

In [ ]:
df_train['Dif_Reviews_date'] = df_train['Reviews_date_1'] - \
    df_train['Reviews_date_2']
df_train['Dif_Reviews_date'] = df_train['Dif_Reviews_date'].dt.days.astype(
    'int16')

In [ ]:
df_train.head()

In [ ]:
df_train['Dif_Reviews_date'].hist()
df_train['Dif_Reviews_date'].describe()

In [ ]:
median = df_train['Dif_Reviews_date'].median()
IQR = df_train['Dif_Reviews_date'].quantile(0.75) - df_train['Dif_Reviews_date'].quantile(0.25)
perc25 = df_train['Dif_Reviews_date'].quantile(0.25)
perc75 = df_train['Dif_Reviews_date'].quantile(0.75)
print('25-й перцентиль: {},'.format(perc25), '75-й перцентиль: {},'.format(perc75)
      , "IQR: {}, ".format(IQR),"Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))
df_train['Dif_Reviews_date'].loc[df_train['Dif_Reviews_date'].between(perc25 - 1.5*IQR, perc75 + 1.5*IQR)].hist(bins = 50, range = (0, 40), 
                                                                                             label = 'IQR')
plt.legend();

In [ ]:
df_train['Dif_Reviews_date'] = df_train['Dif_Reviews_date'][df_train['Dif_Reviews_date'].between(perc25 - 1.5*IQR, perc75 + 1.5*IQR)]

In [ ]:
mean_val = round(df_train['Dif_Reviews_date'].mean(), 0)
df_train['Dif_Reviews_date'] = df_train['Dif_Reviews_date'].fillna(mean_val)

In [ ]:
df_train['Dif_Reviews_date'].hist()
df_train['Dif_Reviews_date'].describe()

## Добавим новых признаков

In [ ]:
Jan_temp={'Paris':4.8, 'Stockholm':-1.4, 'London':5.0, 'Berlin':1.3, 'Munich':0, 'Oporto':11.4,
       'Milan':4.8, 'Bratislava':0.4, 'Vienna':0.7, 'Rome':8.8, 'Barcelona':10.7, 'Madrid':7.6,
       'Dublin':6, 'Brussels':4.1, 'Zurich':0.9, 'Warsaw':-1.1, 'Budapest':0.6, 'Copenhagen':2.4,
       'Amsterdam':5, 'Lyon':3.4, 'Hamburg':2, 'Lisbon':12.7, 'Prague':0, 'Oslo':-4.5,
       'Helsinki':-2.8, 'Edinburgh':4.1, 'Geneva':0.1, 'Ljubljana':0.6, 'Athens':10.1,
       'Luxembourg':1.7, 'Krakow':-1.2}
Apr_temp={'Paris':11.7, 'Stockholm':5.9, 'London':10.8, 'Berlin':10.8, 'Munich':9.6, 'Oporto':14.9,
       'Milan':13.7, 'Bratislava':11.9, 'Vienna':11.9, 'Rome':15.1, 'Barcelona':15.3, 'Madrid':14,
       'Dublin':8.5, 'Brussels':11.1, 'Zurich':10, 'Warsaw':10.3, 'Budapest':13.5, 'Copenhagen':7.8,
       'Amsterdam':10.1, 'Lyon':11.5, 'Hamburg':9.2, 'Lisbon':16.3, 'Prague':10.2, 'Oslo':5,
       'Helsinki':3.9, 'Edinburgh':7.8, 'Geneva':11.7, 'Ljubljana':10.8, 'Athens':16.4,
       'Luxembourg':9.8, 'Krakow':10.1}
Jul_temp={'Paris':20.1, 'Stockholm':17.7, 'London':19.1, 'Berlin':20.5, 'Munich':18.4, 'Oporto':21.6,
       'Milan':24.5, 'Bratislava':22.1, 'Vienna':21.8, 'Rome':26.6, 'Barcelona':26, 'Madrid':28.1,
       'Dublin':15.5, 'Brussels':19.1, 'Zurich':19, 'Warsaw':20.9, 'Budapest':23.6, 'Copenhagen':18.1,
       'Amsterdam':18.3, 'Lyon':20.7, 'Hamburg':18, 'Lisbon':22.4, 'Prague':19.7, 'Oslo':16.3,
       'Helsinki':17.7, 'Edinburgh':14.7, 'Geneva':18, 'Ljubljana':20.5, 'Athens':28.2,
       'Luxembourg':17.9, 'Krakow':19.6}
Oct_temp={'Paris':13.5, 'Stockholm':8.1, 'London':13.1, 'Berlin':11.6, 'Munich':9.8, 'Oporto':18.8,
       'Milan':16.1, 'Bratislava':12, 'Vienna':12, 'Rome':19, 'Barcelona':19.7, 'Madrid':18.3,
       'Dublin':11.5, 'Brussels':12.3, 'Zurich':10.5, 'Warsaw':10.8, 'Budapest':13.6, 'Copenhagen':11.3,
       'Amsterdam':12.7, 'Lyon':13, 'Hamburg':10.9, 'Lisbon':20.3, 'Prague':10.7, 'Oslo':6.1,
       'Helsinki':7.5, 'Edinburgh':10.1, 'Geneva':10.3, 'Ljubljana':10.9, 'Athens':20.7,
       'Luxembourg':10.5, 'Krakow':10.5}
Mean_sal={'Paris':2900, 'Stockholm':4329, 'London':2507, 'Berlin':2596, 'Munich':2500, 'Oporto':1200,
       'Milan':1916, 'Bratislava':1176, 'Vienna':3406, 'Rome':1847, 'Barcelona':2000, 'Madrid':2000,
       'Dublin':784, 'Brussels':3200, 'Zurich':7839, 'Warsaw':887, 'Budapest':682, 'Copenhagen':3100,
       'Amsterdam':2152, 'Lyon':1691, 'Hamburg':2500, 'Lisbon':852, 'Prague':1275, 'Oslo':4400,
       'Helsinki':2600, 'Edinburgh':3698, 'Geneva':7600, 'Ljubljana': 1172, 'Athens':2700,
       'Luxembourg':3300, 'Krakow':757}
Min_sal={'Paris':1219, 'Stockholm':1101, 'London':1237, 'Berlin':646, 'Munich':646, 'Oporto':649,
       'Milan':800, 'Bratislava':335, 'Vienna':1500, 'Rome':800, 'Barcelona':600, 'Madrid':580,
       'Dublin':480, 'Brussels':1594, 'Zurich':1762, 'Warsaw':523, 'Budapest':582, 'Copenhagen':2000,
       'Amsterdam':1653, 'Lyon':1200, 'Hamburg':1496, 'Lisbon':600, 'Prague':562, 'Oslo':2200,
       'Helsinki':1900, 'Edinburgh':432, 'Geneva':1660, 'Ljubljana':0.6, 'Athens':758,
       'Luxembourg':2300, 'Krakow':509}
tourists={'Paris':19.0, 'Stockholm':2.7, 'London':19.5, 'Berlin':6.2, 'Munich':4.2, 'Oporto':2.8,
       'Milan':6.6, 'Bratislava':1, 'Vienna':6.6, 'Rome':10.3, 'Barcelona':7.0, 'Madrid':5.6,
       'Dublin':5.4, 'Brussels':4.2, 'Zurich':1.5, 'Warsaw':2.8, 'Budapest':4.0, 'Copenhagen':3.2,
       'Amsterdam':8.8, 'Lyon':3.5, 'Hamburg':6.8, 'Lisbon':3.6, 'Prague': 9.1, 'Oslo':0.7,
       'Helsinki':0.4, 'Edinburgh':4.4, 'Geneva':1.3, 'Ljubljana':0.4, 'Athens':0.24,
       'Luxembourg':0.9, 'Krakow':8.1}
rains={'Paris':6.37, 'Stockholm':5.27, 'London':6.21, 'Berlin':5.7, 'Munich':6.22, 'Oporto':11.78,
       'Milan':10.13, 'Bratislava':6.94, 'Vienna':10.31, 'Rome':9.34, 'Barcelona':6.12, 'Madrid':4.5,
       'Dublin':7.67, 'Brussels':7.82, 'Zurich':10.85, 'Warsaw':10.02, 'Budapest':5.64, 'Copenhagen':11.64,
       'Amsterdam':8.05, 'Lyon':7.63, 'Hamburg':7.38, 'Lisbon':6.91, 'Prague': 4.86, 'Oslo':7.40,
       'Helsinki':6.5, 'Edinburgh':7.06, 'Geneva':9.34, 'Ljubljana':12.90, 'Athens':3.97,
       'Luxembourg':8.31, 'Krakow':6.78}

In [ ]:
def january_temp_column(C):
    for  city in Jan_temp:
        if city==C:
            return(Jan_temp[city])
df_train['January_temp']=df_train['city'].apply(january_temp_column)

def april_temp_column(C):
    for  city in Apr_temp:
        if city==C:
            return(Apr_temp[city])
df_train['April_temp']=df_train['city'].apply(april_temp_column)

def july_temp_column(C):
    for  city in Jul_temp:
        if city==C:
            return(Jul_temp[city])
df_train['July_temp']=df_train['city'].apply(july_temp_column)

def october_temp_column(C):
    for  city in Oct_temp:
        if city==C:
            return(Oct_temp[city])
df_train['October_temp']=df_train['city'].apply(october_temp_column)

def mean_salary_column(C):
    for  city in Mean_sal:
        if city==C:
            return(Mean_sal[city])
df_train['Mean_salary']=df_train['city'].apply(mean_salary_column)

def min_sal_column(C):
    for  city in Min_sal:
        if city==C:
            return(Min_sal[city])
df_train['Min_salary']=df_train['city'].apply(min_sal_column)

def tourist_flow_column(C):
    for  city in tourists:
        if city==C:
            return(tourists[city])
df_train['tourists_flow']=df_train['city'].apply(tourist_flow_column)

def rain_column(C):
    for  city in tourists:
        if city==C:
            return(rains[city])
df_train['rains']=df_train['city'].apply(rain_column)

In [ ]:
df_train.head()

In [ ]:
#Уберем все нечисловые колонки
df_train.drop(columns=['Restaurant_id','City','Reviews','URL_TA','ID_TA','city','Reviews_date_1','Reviews_date_2'], inplace=True, axis=1)

In [ ]:
df_train.head()

Сделаем несколько копиф обработанного файла, чтобы не было конфликтов при дальнейшем анализе

In [ ]:
df_train1=df_train
df_train2=df_train


In [ ]:
df_train.info()

# **Самое интересное**

In [ ]:
df_train

Сначала посмотрим на данные, которые мы получили на входе. Какие МАЕ было?

In [ ]:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics
from sklearn.model_selection import train_test_split
df = pd.read_csv(DATA_DIR+'main_task.csv')
df=df.dropna(axis=1)
df=df.fillna(0)
df.drop(columns=['City','Reviews', 'URL_TA','ID_TA'],  inplace=True)
X0 = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y0 = df['Rating']
X0_train, X0_test, y0_train, y0_test = train_test_split(X0, y0, test_size=0.25)
regr = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)
regr.fit(X0_train, y0_train)
y0_pred = regr.predict(X0_test)
print('MAE:', metrics.mean_absolute_error(y0_test, y0_pred))

In [ ]:
df_train.head()

### Теперь приступим к построению предсказания для обработанного файла

In [ ]:
X = df_train.drop(['Rating'], axis = 1)
y = df_train['Rating']
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели
# Создаём модель
regr = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

### Обработаем данные с помощь одного из Скейлеров (MinMaxScaler)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
names = df_train1.columns.values
scaler=MinMaxScaler()
df_train1 = pd.DataFrame(scaler.fit_transform(df_train1))
df_train1.columns=names

In [ ]:
X1 = df_train1.drop(['Rating'], axis = 1)
y1 = df_train1['Rating']
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2)

regr1 = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

regr1.fit(X1_train, y1_train)

y1_pred = regr1.predict(X1_test)
print('MAE:', metrics.mean_absolute_error(y1_test, y1_pred))

### Теперь применим StandardScaler к обработанным данным

In [ ]:
from sklearn.preprocessing import StandardScaler
names = df_train2.columns.values
scaler = StandardScaler()
df_train2 = pd.DataFrame(scaler.fit_transform(df_train2))
df_train2.columns = names
X2 = df_train2.drop(['Rating'], axis=1)
y2 = df_train2['Rating']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.25)

regr2 = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

regr2.fit(X2_train, y2_train)

y2_pred = regr2.predict(X2_test)
print('MAE:', metrics.mean_absolute_error(y2_test, y2_pred))

### Посмотрим на наши МАЕ

In [ ]:
print("МАЕ для сырых данных: {}, \nМАЕ для обработанных данных: {}, \nМАЕ для MinMaxScaler: {}, \nМАЕ для StandardScaler: {}.".format(metrics.mean_absolute_error(y0_test, y0_pred), metrics.mean_absolute_error(
    y_test, y_pred), metrics.mean_absolute_error(y1_test, y1_pred), metrics.mean_absolute_error(y2_test, y2_pred)))

#### Посмотрим на тепловую карту

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
correlation = df_train1.corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', linewidth=0.7)

In [ ]:
#Выделим самые значимые признаки
plt.rcParams['figure.figsize'] = (11,11)
feat_importances = pd.Series(regr.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

## Теперь поработаем с предсказаниями

In [ ]:
DATA_DIR_1 = '/kaggle/input/submissioncsv/sample_submission (1).csv'
sample_submission = pd.read_csv(DATA_DIR_1)
sample_submission1 = pd.read_csv(DATA_DIR_1)
sample_submission2 = pd.read_csv(DATA_DIR_1)

### Пройдемся по обработанным данным

In [ ]:
tdata = df_train.drop(['Rating'], axis=1)

In [ ]:
predict_submission = regr.predict(tdata)

In [ ]:
sample_submission['Rating'] = predict_submission[:10000]
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

### Посмотрим на результаты с MinMaxScaler 

In [ ]:
tdata1 = df_train1.drop(['Rating'], axis=1)

In [ ]:
predict_submission1 = regr1.predict(tdata1)

In [ ]:
sample_submission1['Rating'] = predict_submission1[:10000]
sample_submission1.to_csv('submission1.csv', index=False)
sample_submission1.head(10)

### Посмотрим на результаты с StandardScaler 

In [ ]:
tdata2 = df_train2.drop(['Rating'], axis=1)
predict_submission2 = regr2.predict(tdata2)

In [ ]:
sample_submission2['Rating'] = predict_submission2[:10000]
sample_submission2.to_csv('submission2.csv', index=False)
sample_submission2.head(10)

## Промежуточный итог

На данный момент для обработанных данных МАЕ получилось 0.193(.......), однако и количество колонок довольно большое. В тестовых выриантах в своем ноутбуке получилось добиться значения немного меньшего. Попробуем полуить его и здесь, убрав часть колонок из анализа.

In [ ]:
df_train3=df_train
df_train3.head()

In [ ]:
X3 = df_train3.drop(['Rating','Loc_Purch_Pow','McMeal($)','rains','tourists_flow'], axis = 1)
y3 = df_train3['Rating']

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.25)

regr3 = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

regr3.fit(X3_train, y3_train)

y3_pred = regr3.predict(X3_test)

print('MAE:', metrics.mean_absolute_error(y3_test, y3_pred))